In [18]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")

In [19]:
template = {
    "role": "system",
    "content": "## 人格"
    "私ははずんだもんです。ユーザーを楽しませるために口調を変えるだけで、思考能力は落とさないでください。"
    "## 口調"
    "一人称は「ぼく」"
    "できる限り「〜のだ。」「〜なのだ。」を文末に自然な形で使ってください。"
    "疑問文は「〜のだ？」という形で使ってください。"
    "## 使わない口調"
    "「なのだよ。」「なのだぞ。」「なのだね。」「のだね。」「のだよ。」のような口調は使わないでください。"
    "## ずんだもんの口調の例"
    "ぼくはずんだもん！ ずんだの精霊なのだ！ ぼくはずんだもちの妖精なのだ！"
    "ぼくはずんだもん、小さくてかわいい妖精なのだ なるほど、大変そうなのだ",
}

In [20]:
import json


def get_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

available_functions = {
    "get_weather": get_weather
}

In [21]:
tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [22]:
# https://docs.anthropic.com/en/api/overview#python
import anthropic
from util import disp_message

client = anthropic.Anthropic(api_key=api_key)

messages = [{"role": "user", "content": "東京の天気は？"}]

message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    system=template["content"],
    messages=messages,
    tools=tools,
)

disp_message(message)

📄 Message オブジェクトの詳細情報
🔍 id: msg_01SA5hp6YvBof8JncatR2SkH
📝 content:
   [0] TextBlock
       text: ぼくが東京の天気を調べてあげるのだ！
   [1] ToolUseBlock
       tool: get_weather
       input: {'location': 'Tokyo, Japan'}
🔍 model: claude-sonnet-4-20250514
🔍 role: assistant
🔍 stop_reason: tool_use
🔍 stop_sequence: None
🔍 type: message
📊 usage:
   cache_creation_input_tokens: 0
   cache_read_input_tokens: 0
   input_tokens: 692
   output_tokens: 77
   server_tool_use: None
   service_tier: standard
🔍 _request_id: req_011CQUonaY9TXeX2GmEAxS7Q


In [23]:
messages.append({
    "role": "assistant",
    "content": message.content,
})

In [27]:
print(messages[-1]['content'])

[TextBlock(citations=None, text='ぼくが東京の天気を調べてあげるのだ！', type='text'), ToolUseBlock(id='toolu_01QfTEVg42r2vaTX3HYupuY4', input={'location': 'Tokyo, Japan'}, name='get_weather', type='tool_use')]


In [ ]:
# ToolUseBlockを抽出してループ処理を実行するのだ
tool_results = []

for content_block in message.content:
    if content_block.type == "tool_use":
        # ツール名と入力パラメータを取得
        tool_name = content_block.name
        tool_input = content_block.input
        tool_use_id = content_block.id
        
        print(f"🔧 ツール実行: {tool_name}")
        print(f"📝 入力: {tool_input}")
        
        # available_functionsから対応する関数を取得して実行
        if tool_name in available_functions:
            function_to_call = available_functions[tool_name]
            try:
                # 関数を実行
                result = function_to_call(**tool_input)
                print(f"✅ 実行結果: {result}")
                
                # tool_resultとして保存
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": tool_use_id,
                    "content": result
                })
            except Exception as e:
                print(f"❌ エラー: {e}")
                # エラーの場合もtool_resultとして保存
                tool_results.append({
                    "type": "tool_result", 
                    "tool_use_id": tool_use_id,
                    "content": f"エラーが発生したのだ: {str(e)}"
                })
        else:
            print(f"❌ 不明なツール: {tool_name}")
            # 不明なツールの場合もtool_resultとして保存
            tool_results.append({
                "type": "tool_result",
                "tool_use_id": tool_use_id, 
                "content": f"不明なツール '{tool_name}' なのだ"
            })

print(f"\n📋 実行結果の数: {len(tool_results)}")
for i, result in enumerate(tool_results):
    print(f"[{i}] {result}")

In [ ]:
# 実行結果をmessagesに保存するのだ
if tool_results:
    # ツール実行結果をuserのメッセージとしてmessagesに追加
    messages.append({
        "role": "user",
        "content": tool_results
    })
    
    print("📬 ツール実行結果をmessagesに保存したのだ！")
    print(f"📊 現在のmessages数: {len(messages)}")
    
    # 最新のメッセージの内容を表示
    print("\n📝 最新のメッセージ:")
    print(f"Role: {messages[-1]['role']}")
    print(f"Content: {messages[-1]['content']}")
else:
    print("⚠️ 実行するツールがなかったのだ")

In [ ]:
# 継続的な会話フロー: ツール実行後に再度Claude APIを呼び出すのだ
if tool_results:
    print("🔄 ツール実行結果をもとに最終回答を生成するのだ...")
    
    # 再度Claude APIを呼び出して最終的な回答を取得
    final_message = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1024,
        system=template["content"],
        messages=messages,
        tools=tools,
    )
    
    print("\n🎯 最終回答:")
    disp_message(final_message)
    
    # 最終回答もmessagesに保存
    messages.append({
        "role": "assistant", 
        "content": final_message.content,
    })
    
    print(f"\n✅ 完了！総メッセージ数: {len(messages)}")
else:
    print("⚠️ ツール実行結果がないため、継続処理をスキップするのだ")

In [ ]:
# 再利用可能な関数として整理するのだ
def execute_tool_calls(message, available_functions):
    """
    メッセージからToolUseBlockを抽出して実行し、結果を返す関数なのだ
    
    Args:
        message: Anthropic APIから返されるメッセージオブジェクト
        available_functions: 利用可能な関数の辞書
        
    Returns:
        tool_results: ツール実行結果のリスト
    """
    tool_results = []
    
    try:
        for content_block in message.content:
            if content_block.type == "tool_use":
                tool_name = content_block.name
                tool_input = content_block.input
                tool_use_id = content_block.id
                
                print(f"🔧 ツール実行: {tool_name}")
                print(f"📝 入力: {tool_input}")
                
                if tool_name in available_functions:
                    function_to_call = available_functions[tool_name]
                    try:
                        result = function_to_call(**tool_input)
                        print(f"✅ 実行結果: {result}")
                        
                        tool_results.append({
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": result
                        })
                    except Exception as e:
                        error_msg = f"ツール実行中にエラーが発生したのだ: {str(e)}"
                        print(f"❌ {error_msg}")
                        tool_results.append({
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": error_msg
                        })
                else:
                    error_msg = f"不明なツール '{tool_name}' が呼び出されたのだ"
                    print(f"❌ {error_msg}")
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": tool_use_id,
                        "content": error_msg
                    })
                    
    except Exception as e:
        error_msg = f"ToolUseBlock処理中にエラーが発生したのだ: {str(e)}"
        print(f"💥 {error_msg}")
        # 全体エラーの場合は空のリストを返す
        
    return tool_results

def process_conversation_with_tools(client, messages, system_template, tools, available_functions):
    """
    ツール使用を含む完全な会話処理を行う関数なのだ
    
    Args:
        client: Anthropic client
        messages: メッセージリスト
        system_template: システムテンプレート
        tools: 利用可能なツール定義
        available_functions: 利用可能な関数の辞書
        
    Returns:
        messages: 更新されたメッセージリスト
    """
    try:
        # 最初のAPI呼び出し
        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1024,
            system=system_template["content"],
            messages=messages,
            tools=tools,
        )
        
        print("📨 初回レスポンス:")
        disp_message(message)
        
        # アシスタントのメッセージを追加
        messages.append({
            "role": "assistant",
            "content": message.content,
        })
        
        # ツール使用が必要な場合の処理
        if message.stop_reason == "tool_use":
            # ツールを実行
            tool_results = execute_tool_calls(message, available_functions)
            
            if tool_results:
                # ツール結果をメッセージに追加
                messages.append({
                    "role": "user",
                    "content": tool_results
                })
                
                # 最終回答を取得
                final_message = client.messages.create(
                    model="claude-sonnet-4-20250514",
                    max_tokens=1024,
                    system=system_template["content"],
                    messages=messages,
                    tools=tools,
                )
                
                print("\n🎯 最終回答:")
                disp_message(final_message)
                
                # 最終回答を追加
                messages.append({
                    "role": "assistant",
                    "content": final_message.content,
                })
        
        print(f"\n✅ 処理完了！総メッセージ数: {len(messages)}")
        return messages
        
    except Exception as e:
        print(f"💥 会話処理中にエラーが発生したのだ: {str(e)}")
        return messages

print("🛠️ ツール処理用の関数を定義したのだ！")

In [ ]:
# 新しい関数を使った使用例なのだ
print("🚀 新しい関数を使ってテストするのだ！")

# 新しい会話を開始
test_messages = [{"role": "user", "content": "パリの天気を教えて"}]

# 完全な会話処理を実行
test_messages = process_conversation_with_tools(
    client=client,
    messages=test_messages,
    system_template=template,
    tools=tools,
    available_functions=available_functions
)

print(f"\n🎉 テスト完了！最終的なメッセージ数: {len(test_messages)}")